In [1]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install torch torchvision opencv-python numpy

  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\hllqkb\appdata\local\temp\pip-req-build-94pf9p9z
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36875 sha256=a153e7033e381fb5d23835e4809a29f11a1a6a8d0c9d1320dd961ab42bae2671
  Stored in directory: C:\Users\hllqkb\AppData\Local\Temp\pip-ephem-wheel-cache-xbt9md5k\wheels\15\d7\bd\05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\hllqkb\AppData\Local\Temp\pip-req-build-94pf9p9z'


In [2]:
# Create SAM predictor
import os
import cv2
import numpy as np
import torch
from segment_anything import SamPredictor,sam_model_registry
import urllib.request

def wget(url, filename=None):
    """
    下载文件到当前文件夹，类似wget命令。
    url: 文件下载链接
    filename: 保存的文件名（可选），不填则自动取url最后一段
    """
    if filename is None:
        filename = url.split('/')[-1]
    print(f"Downloading {url} to {filename} ...")
    urllib.request.urlretrieve(url, filename)
    print("Download complete.")

# Download the model if not exists
model_path = './sam_vit_b_01ec64.pth'
if not os.path.exists(model_path):
    wget("https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth", model_path)
# Load the predictor
sam=sam_model_registry["vit_b"](checkpoint=model_path)
predictor=SamPredictor(sam)

Download complete.


c:\Users\hllqkb\anaconda3\envs\pytorch-gpu\Lib\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [5]:
# load image and select x,y coordinates
image_path = './test.jpg'
if not os.path.exists(image_path):
  # !wget https://utils-computervisiondeveloper.s3.amazonaws.com/media/public/test.jpg
  wget("https://utils-computervisiondeveloper.s3.amazonaws.com/media/public/test.jpg", image_path)

x = 528
y = 606

image = cv2.imread(image_path)

Download complete.


In [6]:
# Use sam predictor on (image, x, y) to get location of subject
predictor.set_image(image)
masks,scores,logits = predictor.predict(
    point_coords=np.array([[x, y]]),
    point_labels=np.array([1]),
    multimask_output=True
)
C,H,W = masks.shape
result_mask = np.zeros((H, W), dtype=bool)

for j in range(C):
  result_mask |= masks[j, :, :]

result_mask = result_mask.astype(np.uint8)
     

In [10]:
# remove background
alpha_channel = np.ones(image.shape[:2], dtype=np.uint8) * 255
alpha_channel[result_mask == 0] = 0
bg_removed = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
bg_removed[:, :, 3] = alpha_channel
cv2.imwrite('bg_removed.png', bg_removed)

True

In [14]:
# wrap it up as a function 
# wrap it up as a function
import base64
import os

from segment_anything import SamPredictor, sam_model_registry
import cv2
import numpy as np


model_path = './sam_vit_b_01ec64.pth'
if not os.path.exists(model_path):
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

sam = sam_model_registry["vit_b"](checkpoint=model_path)
predictor = SamPredictor(sam)


def remove_background(image_base64_encoding, x, y):

  image_bytes = base64.b64decode(image_base64_encoding)

  image = cv2.imdecode(np.frombuffer(image_bytes, dtype=np.uint8), cv2.IMREAD_COLOR)

  predictor.set_image(image)

  masks, scores, logits = predictor.predict(
                                    point_coords=np.asarray([[x, y]]),
                                    point_labels=np.asarray([1]),
                                    multimask_output=True
                                )

  C, H, W = masks.shape

  result_mask = np.zeros((H, W), dtype=bool)

  for j in range(C):
    result_mask |= masks[j, :, :]

  result_mask = result_mask.astype(np.uint8)

  alpha_channel = np.ones(result_mask.shape, dtype=result_mask.dtype) * 255

  alpha_channel[result_mask == 0] = 0

  result_image = cv2.merge((image, alpha_channel))

  _, result_image_bytes = cv2.imencode('.png', result_image)

  result_image_bytes = result_image_bytes.tobytes()

  result_image_bytes_encoded_base64 = base64.b64encode(result_image_bytes).decode('utf-8')

  return result_image_bytes_encoded_base64
     

import cv2
import matplotlib.pyplot as plt


image_path = './test.jpg'
if not os.path.exists(image_path):
  !wget https://utils-computervisiondeveloper.s3.amazonaws.com/media/public/test.jpg

x = 528
y = 606

image = cv2.imread(image_path)

_, image_bytes = cv2.imencode('.png', image)

image_bytes = image_bytes.tobytes()

image_bytes_encoded_base64 = base64.b64encode(image_bytes).decode('utf-8')

result_image = remove_background(image_bytes_encoded_base64, x, y)

result_image_bytes = base64.b64decode(result_image)

result_image = cv2.imdecode(np.frombuffer(result_image_bytes, dtype=np.uint8), cv2.IMREAD_UNCHANGED)

plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGRA2RGBA))
plt.show()

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


FileNotFoundError: [Errno 2] No such file or directory: './sam_vit_b_01ec64.pth'